### Postprocessing Results
Analyses were run off in 50 'buckets' of 1,000 (load cases).  This was done to enable the analsyes to be run in parallel across several machines. Timetraces for 6 parameters were generated for each analysis. These timetraces need to post-processed to get the max range for each parameter.  A summary dataframe for each bucket (1,000 files) will be stored in the analysis results sub-folder.  

In [113]:
import glob as gb
import ntpath # Equivalent of R's base (remove path)
import pandas as pd

In [114]:
# Results will be processed a the become available so let's process one at a time
# Specify the folder where the results are located
BUCKET = 'bucket33'
#local_dir = f'/Users/jasonpayne/Documents/slu_res/{BUCKET}/'
local_dir = f'C:\\Flexcom_Analysis\\_slug_project\\{BUCKET}\\'
plot_files = gb.glob(f'{local_dir}*.mplt')

In [115]:
# Create a list of load case references that are covered by the timetrace files
lc_files = []
for file_p in plot_files:
    # Remove the last 8 chars from the plot file name (this will be the load case ref.)
    lc_files.append(ntpath.basename(file_p[:len(file_p) - 8]))

# Remove duplicates & sort
lc_files = list(dict.fromkeys(lc_files))
lc_files.sort()

In [116]:
# Create empty lists of take processed results
load_cases = []
out_verDisp_mm_n5 = []
out_oopDisp_mm_n5 = []
out_oopDisp_mm_n2 = []
out_endBend_kNm_n6 = []
out_supReac_kN_n3 = []
out_supReac_kN_n4 = []

# Loop through and process each load case & each plot file in each load case
for lc in lc_files:
    load_cases.append(lc)
    for tt in ['T1', 'T3', 'T4', 'T5', 'T6', 'T7']:
        # Reconstruct the filename from the load case reference
        file_p = f'{lc}.{tt}.mplt'
        
        # Read the time trace file (skip header & note unusual delimeter)
        plot_raw = pd.read_csv(f'{local_dir}{file_p}', skiprows=12, delimiter='"')
        dat_dt = {'time':plot_raw.iloc[:,3].dropna(), 'value':plot_raw.iloc[:,5].dropna()}
        dat_df = pd.DataFrame(dat_dt)

        # Breakdown of types of timetrace
        motions = ['T1', 'T2', 'T3', 'T4']
        moments = ['T5', 'T6']
        reactin = ['T7']

        # Apply some unit conversions depending on type
        # Use 'mm' for displacement ranges
        if (any(x in file_p for x in motions)):
            dat_df.value = dat_df.value * 1000

        elif (any(x in file_p for x in moments)):
            dat_df.value = dat_df.value / 1000

        elif (any(x in file_p for x in reactin)):
            dat_df.value = dat_df.value / 1000

        # Calculate the range
        dat_range = dat_df.value.max() - dat_df.value.min()
        #print(file_p[:len(file_p) - 8])
        #print(f'Max: {round(dat_df.value.max(), 1)}, Min: {round(dat_df.value.min(), 1)}') 
        #print('Range: ', round(dat_range, 1), '\n')

        # Collect the results
        if 'T1' in file_p:
            out_verDisp_mm_n5.append(round(dat_range, 2))
        elif 'T3' in file_p:
            out_oopDisp_mm_n5.append(round(dat_range, 2))
        elif 'T4' in file_p:
            out_oopDisp_mm_n2.append(round(dat_range, 2))
        elif 'T5' in file_p:
            out_endBend_kNm_n6.append(round(dat_range, 2))
        elif 'T6' in file_p:
            out_supReac_kN_n3.append(round(dat_range, 2)) 
        elif 'T7' in file_p:
            out_supReac_kN_n4.append(round(dat_range, 2))     
    

In [117]:
# Now compile the results in a single dataframe
# ver: Vertical
# oop: Out of plane
# disp: Displacement
# Reac: Reaction
res_df = pd.DataFrame(load_cases, columns = ['load_case'])
res_df['out_oopDisp_mm_n2'] = out_oopDisp_mm_n2
res_df['out_supReac_kN_n3'] = out_supReac_kN_n3
res_df['out_supReac_kN_n4'] = out_supReac_kN_n4
res_df['out_verDisp_mm_n5'] = out_verDisp_mm_n5
res_df['out_oopDisp_mm_n5'] = out_oopDisp_mm_n5
res_df['out_endBend_kNm_n6'] = out_endBend_kNm_n6
res_df

,load_case,out_oopDisp_mm_n2,out_supReac_kN_n3,out_supReac_kN_n4,out_verDisp_mm_n5,out_oopDisp_mm_n5,out_endBend_kNm_n6
0,32001_sd654_sl8_pd171_pl12_sv8_bp13_ba770_bp55,190.40,15.98,54.17,259.60,196.70,211.81
1,32002_sd695_sl12_pd286_pl12_sv6_bp11_ba850_bp56,199.48,15.77,55.72,252.13,187.29,212.88
2,32003_sd666_sl10_pd280_pl10_sv8_bp12_ba600_bp57,146.85,13.49,47.45,204.33,140.23,188.09
3,32004_sd642_sl9_pd225_pl11_sv6_bp12_ba700_bp64,174.37,14.30,51.27,214.94,176.61,170.84
4,32005_sd778_sl12_pd344_pl9_sv9_bp12_ba750_bp51,214.44,16.60,56.95,260.73,241.57,271.83
...,...,...,...,...,...,...,...
995,32996_sd781_sl12_pd144_pl6_sv8_bp10_ba610_bp52,221.56,17.01,51.90,260.88,272.05,218.05
996,32997_sd786_sl9_pd250_pl6_sv7_bp11_ba630_bp49,153.81,14.20,51.34,222.06,173.27,175.32
997,32998_sd759_sl10_pd346_pl12_sv10_bp12_ba800_bp45,198.54,16.84,59.46,252.14,187.11,250.35
998,32999_sd780_sl11_pd216_pl8_sv9_bp11_ba860_bp51,209.73,17.61,60.61,249.31,229.98,268.08


In [118]:
# Write the results as a csv
res_df.to_csv(f'{BUCKET}_results.csv', index=False)